In [2]:
#encoding=utf-8
# 基本部件的引用声明
import sys
import re
import codecs
import os
import shutil
import jieba
import jieba.analyse
import graphlab
import numpy as np
from array import array
from collections import Counter

### 注意我们这里需要安装好的依赖有Graphlab，numpy，jieba
* Numpy和Jieba直接pip即可
* Graphlab是商业软件，需要授权
    * PIP安装时用此命令：(括号内可选)
         * (sudo) pip install (--user) --upgrade --no-cache-dir https://get.graphlab.com/GraphLab-Create/2.0.1/ouyf5@mail2.sysu.edu.cn/F822-FCB4-973D-5C49-44E0-430C-BC81-4EA3/GraphLab-Create-License.tar.gz

### 首先是从文件中的读取以及分词


此处用的是 *Jieba* 中文分词器，在这里我们将中文文本分词之后以空格分隔各词然后输出到目标文件夹

函数定义中需传入 **CSV文件的绝对地址** 

格式要求：
   * CSV中字串以UTF-8格式编码，CSV文件格式为id,content(需要有Header Row)
   * 传入目标文件为绝对地址
   * 获得的数据为SFrame格式，column与CSV格式一致
   * SFrame参考文档：https://turi.com/products/create/docs/generated/graphlab.SFrame.html?highlight=sframe

#### 首先是添加停词表（Stop Words List）
* 加入停词表能够把常用的词语（通常是介词、连接词等）去掉，提高分类的准确性
* get_stop_words 这个方法从文件中读取停词列表，然后返回一个 **停词集合**

In [3]:
def read_from_file(path):
    with open(path,"r") as fp:
        words = fp.read()
    return words

In [4]:
def get_stop_words(filepath):
    words = read_from_file(filepath)
    result = jieba.cut(words)
    new_words = []
    for r in result:
        new_words.append(r)
    return set(new_words)

In [5]:
def remove_stop_words(words,stop_words_set):
    new_words = []
    for w in words:
        if w not in stop_words_set:
            new_words.append(w)
    return new_words

#### Checkpoint: 检查上述方法的正确性

In [9]:
#stop_set = get_stop_words('./stop.txt')
#lister = list(stop_set)
#for i in range(10):
#    print lister[i]

weibo = read_and_cut('./data.csv')
#print weibo

train_fake_news = weibo[0:10]
train_fake_weibos = weibo[10:]

weibos_wvec = batch_word_vec_generator(train_fake_weibos['parsed'] , True)
news_wvec = batch_word_vec_generator(train_fake_news['parsed'] , True)


train_fake_news['word_vec'] = news_wvec['word_vec']
train_fake_news['tf_word_vec'] = news_wvec['tf_word_vec']
train_fake_news['assign_weibos'] = batch_NN_finder(train_fake_news['tf_word_vec'] , weibos_wvec['tf_word_vec'] , train_fake_weibos['Id'])
    

This non-commercial license of GraphLab Create is assigned to ouyf5@mail2.sysu.edu.cn and will expire on November 25, 2016. For commercial licensing options, visit https://turi.com/buy/.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.0.1 started. Logging: C:\Users\Tidyzq\AppData\Local\Temp\graphlab_server_1469086529.log.0


Unable to parse line "62,娆ф床鑸�ぉ灞�European Space Agency)灞�暱闊﹀皵绾�Johann-Dietrich Worner)绉帮紝灏界�缇庡浗鍥戒細浠嶅己鐑堝弽瀵癸紝浠栧凡鍐嶆�鎻愬嚭閭��涓�浗鍙備笌鍥介檯绌洪棿绔欓」鐩�殑鎯虫硶銆傞煢灏旂撼鍦ㄦ硶鎭╀集鍕掕埅灞曟湡闂存帴鍙楅噰璁挎椂琛ㄧず锛屼笉搴..."

Unable to parse line "44,鐭ユ儏浜哄＋绉帮紝鏃ユ湰杞�摱闆嗗洟鑲′唤鏈夐檺鍏�徃(SoftBank Group Corp. ,9984.TO)宸茬粡鍚屾剰浠ュ叏鐜伴噾鏂瑰紡鏀惰喘鑻卞浗鑺�墖璁捐�鍏�徃ARM Holdings PLC (ARMH)锛岃�浜ゆ槗浠峰�瓒呰繃320浜跨編鍏冦�璇ョ煡鎯呬汉澹�О锛岄�璁¤繖妗╀氦鏄撳皢浜庡..."

2 lines failed to parse correctly

...

2 lines failed to parse correctly

Finished parsing file G:\workspace\ir\data.csv

Parsing completed. Parsed 98 lines in 0.014013 secs.

Building prefix dict from the default dictionary ...
Dumping model to file cache c:\users\tidyzq\appdata\local\temp\jieba.cache
Loading model cost 1.990 seconds.
Prefix dict has been built succesfully.


RuntimeError: Runtime Exception. Column name text does not exist.

### 下面是正式操作

In [10]:
def read_and_cut(csvpath):
    mydata = graphlab.SFrame(csvpath)
    stop_set = get_stop_words('./stop.txt')
    parsed = []
    linen = {0,1,2,3,4,5,6,7,8,9,19,39,59,99,399,599,999,2999,3999,5999,8888,9999}
    
    for i in range(len(mydata)):
        line = unicode(mydata['text'][i] , "utf-8")
        seglist = jieba.cut(line , cut_all = False)
        parsed_seg = remove_stop_words(seglist , stop_set)
        output = ' '.join(parsed_seg)
        parsed.append(output)
        if (i in linen or i == len(mydata) - 1):
            print "处理 #%d 文章完成" % i
    mydata['parsed'] = parsed
    print "全部完成！"
    return mydata

### 接下来这个函数是用来提取*词数向量*的

* 在分析过程中，可以频繁的调用该函数，需要提供 **原始文本的字串**
* 返回值为一个 *Dict<单词，出现次数>* ,
* 返回的Dict没有依据单词出现的次数排序
* 如果用向量实现并不现实，所以在此处我们使用Dict来实现词数向量的功能

In [11]:
def parse_word_vec(ref_str):
    words = [w.strip() for w in ref_str.split() if w.strip()]
    counter = Counter()
    for w in words:
        counter[w] += 1
    # kv = counter.items()   #这是键值对的列表
    wc_dict = dict(counter)    #这是词典
    #可以用列表做其他事，但只返回词典
    return wc_dict

### 接下来是计算两个词数向量间的距离

* euc_dist 计算两个词数向量之间的欧几里得距离 返回值为浮点数
* cos_dist 计算两个词数向量之间的余弦距离 返回值为浮点数
* jac_dist 计算两个词数向量之间的加权杰卡德距离 返回值为浮点数

注意到，此处只是使用计算距离对词数向量进行Brute-Force计算，很容易把一些常见的词错误的认为是判断两个文本文档相类似的函数的主要依据（如 **"的"** , **"和"** , **"是"**  , **"了"**  , etc）。如果需要计算更精确的数值，应该使用 **TF-IDF** 来进行计算,使用者在使用前应该调用 *calc_tf_idf* 方法。总体来说，即便使用TFIDF处理过，中文文本的聚类正确率还是不是非常的理想。所以可以尽情尝试各种距离（经过初步实验，Euclidean Distance准确度稍微高那么一点点。）

* 注意, *calc_tf_idf* 方法需要使用者传入所有文本（whole documents）的 **SArray词典集合（SArray of dict）** 返回值为SArray的词典集合类型（每行分别为原来单词的dict的SArray集合）。(SArray文档：https://turi.com/products/create/docs/generated/graphlab.SArray.html?highlight=sarray)
* 示例使用方法：
    ```python
        docs['TF_IDF'] = calc_tf_idf(docs['wc_dict'])
    ```
    这样处理之后还是可以通过
    ```python
    docs[docs['dName'] == 'docnamehere'][0]
    ```
    找到某行。

In [12]:
def euc_dist(dict_a , dict_b):
    return graphlab.distances.euclidean(dict_a , dict_b)

def cos_dist(dict_a , dict_b):
    return graphlab.distances.cosine(dict_a , dict_b)

def jac_dist(dict_a , dict_b):
    return graphlab.distances.weighted_jaccard(dict_a , dict_b)

def calc_tf_idf(whole_document_dict):
    return graphlab.text_analytics.tf_idf(whole_document_dict)

### 给某一个特定的新闻找到10个左右Nearnest Neighbour

由于我们在数据库中存放的是微博，所以我们简单的给定 *新闻文档* ，然后给其适配最适合的10个微博。
* 距离此处使用了欧式距离。
* 初始阈值设定为50.

In [23]:
# 不管传入的是否是TF-IDF，要求传的新闻词数向量为Dictionary，whole_document_vec为dict的SFrame
def find_NN(news_vec , whole_document_vec , doc_ids):
    process_vec = graphlab.SFrame()
    #process_vec['original'] = whole_document_vec
    result_value = []
    result_id = []

    for i in range(len(whole_document_vec)):
        cos_val = euc_dist(whole_document_vec[i] , news_vec)
        if cos_val <= 200:
            result_id.append(doc_ids[i])
            result_value.append(cos_val)
    process_vec['similarity'] = result_value
    process_vec['id'] = result_id

    my = process_vec.sort('similarity' , ascending=True)

    if len(process_vec) < 10:
        return my
    else:
        return my[0:10]

### 批量生成工具

In [14]:
def batch_word_vec_generator(dict_set , TF_SELECTOR = False):
        gen_result = graphlab.SFrame()
        word_vec = []
        linen = {0,1,2,3,4,5,6,7,8,9,19,39,59,99,399,599,999,2999,3999,5999,8888,9999}
        for i in range(len(dict_set)):
                temp_vec = parse_word_vec(dict_set[i])
                word_vec.append(temp_vec)
                if i in linen or i == len(dict_set) - 1:
                    print "第 %d 条文章原始词向量计算完成" % i
        gen_result['word_vec'] = word_vec
        print "本数据集的原始词向量全部计算完成"
        if TF_SELECTOR:
                gen_result['tf_word_vec'] = calc_tf_idf(gen_result['word_vec'])
                print "本数据集的TF-IDF词向量全部计算完成"
        return gen_result

In [15]:
def batch_NN_finder(news , weibos , weibo_ids):
    linen = {0,1,2,3,4,5,6,7,8,9,19,39,59,99,399,599,999,2999,3999,5999,8888,9999}
    assignation = []
    for i in range(len(news)):
        temp_nns = find_NN(news[i] , weibos , weibo_ids)
        assignation.append(temp_nns)
        if i in linen or i == len(news) - 1:
            print "第 %d 条新闻的最近邻居计算完成" % (i + 1)
    return assignation

### 函数测试完成，下面是预处理总方法

由于我们需要的是新闻AND ITS 匹配微博
所以本方法只返回一个SFrame，列为 <*新闻ID* , *新闻原文* , *新闻原始词向量* ,*tf词向量* ,*匹配微博* >

注意：**匹配微博**为一个对象，其中含有匹配微博的< *id* , *相似度*>


接受的参数为：
* weibo_src: 微博CSV文件的绝对路径
* news_src: 新闻CSV文件的绝对路径
* TF_SELECTOR: TF-IDF选择子，表示是否需要进行TF词向量计算

In [16]:
#Main process
#Result contains a SFrame that contains news raw word_vec and matching weibo
def pre_process(weibo_src , news_src , TF_SELECTOR=False):
    #分别处理微博和新闻
    weibos = read_and_cut(weibo_src)
    news = read_and_cut(news_src)
    
    #计算词向量和TF词向量
    weibos_wvec = batch_word_vec_generator(weibos['parsed'] , TF_SELECTOR)
    news_wvec = batch_word_vec_generator(news['parsed'] , TF_SELECTOR)
    
    news['word_vec'] = news_wvec['word_vec']
    
    if TF_SELECTOR:
        news['tf_word_vec'] = news_wvec['tf_word_vec']
        news['assign_weibos'] = batch_NN_finder(news['tf_word_vec'] , weibos_wvec['tf_word_vec'] , weibos['id'])
    else:
        news['assign_weibos'] = batch_NN_finder(news['word_vec'] , weibos_wvec['word_vec'] , weibos['id'])
    
    return news,weibos

In [17]:
weibopath = './weibo.csv'
newspath = './news.csv'
TF_SELECTOR = True

weibos = read_and_cut(weibopath)
news = read_and_cut(newspath)

Unable to parse line "http://t.cn/R5aHrrR""

Unable to parse line "http://t.cn/R5cTRG0浣犳�涔堢湅锛�"

Unable to parse line "http://t.cn/R5UWWXX""

Unable to parse line "http://t.cn/R55WxAd""

Unable to parse line "http://t.cn/R55YTvx""

...

Unable to parse line "        绮ゅ叕缃戝畨澶�44010402000035鍙�"

183 lines failed to parse correctly

Finished parsing file G:\workspace\ir\news.csv

Parsing completed. Parsed 1493 lines in 0.056052 secs.

处理 #0 文章完成
处理 #1 文章完成
处理 #2 文章完成
处理 #3 文章完成
处理 #4 文章完成
处理 #5 文章完成
处理 #6 文章完成
处理 #7 文章完成
处理 #8 文章完成
处理 #9 文章完成
处理 #19 文章完成
处理 #39 文章完成
处理 #59 文章完成
处理 #99 文章完成
处理 #399 文章完成
处理 #599 文章完成
处理 #999 文章完成
处理 #1492 文章完成
全部完成！


In [18]:
print "开始计算微博词向量"
weibos_wvec = batch_word_vec_generator(weibos['parsed'] , TF_SELECTOR)
print "微博词向量全部计算完成"
print ""
print "开始计算新闻词向量"
news_wvec = batch_word_vec_generator(news['parsed'] , TF_SELECTOR)
print "新闻词向量全部计算完成"

开始计算微博词向量
第 0 条文章原始词向量计算完成
第 1 条文章原始词向量计算完成
第 2 条文章原始词向量计算完成
第 3 条文章原始词向量计算完成
第 4 条文章原始词向量计算完成
第 5 条文章原始词向量计算完成
第 6 条文章原始词向量计算完成
第 7 条文章原始词向量计算完成
第 8 条文章原始词向量计算完成
第 9 条文章原始词向量计算完成
第 19 条文章原始词向量计算完成
第 39 条文章原始词向量计算完成
第 59 条文章原始词向量计算完成
第 99 条文章原始词向量计算完成
第 399 条文章原始词向量计算完成
第 599 条文章原始词向量计算完成
第 999 条文章原始词向量计算完成
第 2999 条文章原始词向量计算完成
第 3999 条文章原始词向量计算完成
第 4378 条文章原始词向量计算完成
本数据集的原始词向量全部计算完成
本数据集的TF-IDF词向量全部计算完成
微博词向量全部计算完成

开始计算新闻词向量
第 0 条文章原始词向量计算完成
第 1 条文章原始词向量计算完成
第 2 条文章原始词向量计算完成
第 3 条文章原始词向量计算完成
第 4 条文章原始词向量计算完成
第 5 条文章原始词向量计算完成
第 6 条文章原始词向量计算完成
第 7 条文章原始词向量计算完成
第 8 条文章原始词向量计算完成
第 9 条文章原始词向量计算完成
第 19 条文章原始词向量计算完成
第 39 条文章原始词向量计算完成
第 59 条文章原始词向量计算完成
第 99 条文章原始词向量计算完成
第 399 条文章原始词向量计算完成
第 599 条文章原始词向量计算完成
第 999 条文章原始词向量计算完成
第 1492 条文章原始词向量计算完成
本数据集的原始词向量全部计算完成
本数据集的TF-IDF词向量全部计算完成
新闻词向量全部计算完成


In [24]:
news['word_vec'] = news_wvec['word_vec']
news['tf_word_vec'] = news_wvec['tf_word_vec']
print "开始计算最近邻居"
test = news[0:10]

test['assign_weibos'] = batch_NN_finder(test['tf_word_vec'] , weibos_wvec['tf_word_vec'] , weibos['id'])
print "最近邻居全部计算完成"

开始计算最近邻居
第 1 条新闻的最近邻居计算完成
第 2 条新闻的最近邻居计算完成
第 3 条新闻的最近邻居计算完成
第 4 条新闻的最近邻居计算完成
第 5 条新闻的最近邻居计算完成
第 6 条新闻的最近邻居计算完成
第 7 条新闻的最近邻居计算完成
第 8 条新闻的最近邻居计算完成
第 9 条新闻的最近邻居计算完成
第 10 条新闻的最近邻居计算完成
最近邻居全部计算完成


In [26]:
for i in range(len(test)):
    print ""
    print "+++++++++++++++++++++++++++++++++++++++++++++++++++++++"
    print "-------------------------------------------------------"
    print "分析新闻编号： %s." % test['id'][i]
    print "该新闻内容如下："
    print test['text'][i]
    print ""
    print "总共匹配文章数： %d." % len(test['assign_weibos'][i])
    print "以下是匹配列表："
    print "-------------------------------------------------------"
    for ii in range(len(test['assign_weibos'][i])):
        print "--------------------------------------------------------"
        print "匹配微博： #%d." % test['assign_weibos'][i][ii]['id']
        print weibos[weibos['id'] == test['assign_weibos'][i][ii]['id']]['text']
        print "计算所得距离系数： %f." % test['assign_weibos'][i][ii]['similarity']
        print "--------------------------------------------------------"
    print "--------------------------------------------------------"
    print "++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
    print ""


+++++++++++++++++++++++++++++++++++++++++++++++++++++++
-------------------------------------------------------
分析新闻编号： 1.
该新闻内容如下：
　　还有两天，2016全国高考就要开始了！　　就在考生们抓紧复习、最后冲刺时，小编也没有闲着，为学子们制作了一份高考备忘录！这里面包括考试时间、天气情况等等……如果你家里有考生，或者身边有考生，就赶快转给他们吧！！　　一旦在去考场的途中遇到堵车，最好的办法就是出示准考证向民警求助。但小编要说的是，防患于未然是避免问题的最好办法，这两天尽可能提前去熟悉考场周围的交通环境，要做到心中有数，以免出现意外。　　路上发现忘带准考证，如果时间宽裕，可向交警求助帮助取回。时间紧的，要主动向监考教师说明情况，尽量协调先进场，及时给家长打电话将准考证送过来。　　如果考前时间宽裕，可在考点附近购买；如果已经开考了，要立即向监考老师报告，切忌相互之间自行传递。　　如果考生眩晕、脸色苍白，可喝点糖水或牛奶，补充能量；或在监考人员的帮助下，选择适当的场所吸氧，或头低脚高仰卧几分钟，以保证大脑足够供血供氧。家长也可给孩子置配一些常用药品，以备孩子偶有不适时调理所需。一般可准备以下物品：保济丸、藿香正气水、风油精等。　　遇到这样的情况，要停止答卷，学会放松。先闭目养神，做深呼吸，默默数数或伏案休息片刻，让大脑做短暂休息，或闭目做深呼吸3～9次，不考虑答卷上的问题；或者把笔放下、向窗外眺望，借此稳定情绪，缓解紧张心理。　　不刻意入睡，让自己处于无意识的状态，能睡则睡，不强迫自己；可以想一些轻松的事情。有了良性的心理暗示后，情绪就会放松，入睡也就容易了。如果情况特别严重，可以在医生指导下适当用药。　　最后小编要为大家送上的锦囊是“平常心”！希望考生能够以平常心对待考试、社会能够以平常心看待高考。如果说人生是场马拉松，那么高考也许只能算是其中一公里的成绩，保持“配速”、沉着应对，才能取得最终的胜利。只要认识到空无的道理，生命就无往而不胜。生命想透了其实与一个晚期癌症病人无异，什么都不必太过执着，喜欢干点什么事就干点什么事而已。“特朗普现象”其实是美国新教文明进入衰退期以后的一次自救。特朗普和他背后的美国群众，是对内外挑战的坚决应战，是美国文